In [2]:
include("..\\Code_jail\\Master_func.jl")
using JLD

## Fourrier scaling
Before we start, I want to go over scaling something with Fourrier. We'd like to get the Fourrier transform of $g:0 → L$. For this, we want to approximate $g$ through polynomials $T$. Say we have map $φ(x):[0,L]→[-1,1]: x ↦ \frac{2}{L}x-1$ and $φ^{-1}(x):[-1,1]→[0,L]: x ↦ (x+1)\frac{L}{2}$. We get
\begin{align*}
\mathcal{F}(g)[k] &= \int_0^L e^{-ikx}g(x)\mathrm{d}x = \sum_jc_j \int_0^L e^{-ikx}T_j(φ(x))\mathrm{d}x = \frac{L}{2}\sum_jc_j \int_{-1}^1 e^{-ik(x+1)\frac{L}{2}}T_j(x)\mathrm{d}x \\
&= \frac{L}{2}e^{-ik\frac{L}{2}}\sum_jc_j \int_{-1}^1 e^{-ik\frac{L}{2}x}T_j(x)\mathrm{d}x\\
&= \frac{L}{2}e^{-ik\frac{L}{2}}\sum_jc_j \mathcal{F}(T_j)\left[\frac{L}{2}k\right]
\end{align*}

## Fourrier transform for small $k$ 
Let's say we approximated $g₀$ using Chebyshev polynomials $T$. Note then that the Fourrier transform can be acchieved through two ways. We could use the three-term reccurence:
$$
b_{j+1}T_{j+1} = (a_j-x)T_j + b_jT_{j-1} ⇒ b_{j+1}\hat{T}_{j+1} = a_j\hat{T}_j-\partial_k\hat{T}_j + b_j\hat{T}_{j-1}.
$$
We can get $\partial_k\hat{T}_j$, through deriving the three-term recurrence for $\hat{T}_j$
$$ 
b_{j}\partial_k\hat{T}_j = a_{j-1}\partial_k\hat{T}_{j-1}-\partial_{kk}\hat{T}_{j-1} + b_{j-1}\partial_k\hat{T}_{j-2}.
$$
We can go back to $\hat{T}_{0}$ and get all the derivatives. through the three-term recurrence. Luckely,
$$
\hat{T}_{0} = \int_{-1}^1e^{-ikx}\mathrm{d}x = 2\text{sinc}(k) = -2\int_0^1\cos(xk)\mathrm{d}x.
$$
Using this, we stabily compute the $n$th derivative,
$$
\hat{T}^{(n)}_{0} = -2\int_0^1x^n\cos\left(xk+\frac{n\pi}{2}\right)\mathrm{d}x.
$$
Using the three-term recurrence like this is stable for small $k$. It can quickly grow unstable when $k$ grows. Using this approximation as a substitute for the Fourrier transformed function will be nice.

This is too time expensive, even more than just doing the normal quadrature.

## Fourrier transform for large $k$
What if we integrate the Fourrier transform by parts. We get
$$
\int_{-1}^1 e^{-ikx}T(x)\mathrm{d}x = \sum_{j=0}^N (-1)^{j}\left(\frac{i}{k}\right)^{j+1}\left[T^{(j)}(x)e^{-ikx}\right]_{-1}^1 + (-1)^{N+1}\left(\frac{i}{k}\right)^{N+1}\int_{-1}^1T^{(N+1)}(x)e^{-ikx}\mathrm{d}x.
$$
As $k$ is large, we should be able approximate it and just lose the last term.
$$
\int_{-1}^1 e^{-ikx}T(x)\mathrm{d}x = \sum_{j=0}^N (-1)^{j}\left(\frac{i}{k}\right)^{j+1}\left[T^{(j)}(x)e^{-ikx}\right]_{-1}^1
$$